In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))

dataset_numpy = np.loadtxt('data/pima-indians-diabetes3.csv',
                          delimiter=',', skiprows=1)
dataset = torch.from_numpy(dataset_numpy)

In [3]:
X = dataset[:, :-1]
y = dataset[:, -1]

In [4]:
X.shape

torch.Size([768, 8])

In [5]:
y.shape

torch.Size([768])

In [6]:
class Pima(nn.Module):
    def __init__(self):
        super(Pima, self).__init__()
        self.hidden_linear1 = nn.Linear(8, 12)
        self.activation = nn.ReLU()
        self.hidden_linear2 = nn.Linear(12, 8)
        self.hidden_linear3 = nn.Linear(8, 1)
        self.output_activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.hidden_linear1(x)
        x = self.activation(x)
        x = self.hidden_linear2(x)
        x = self.activation(x)
        x = self.hidden_linear3(x)
        x = self.output_activation(x)
        
        return x
    
    def predict(self, x):
        pred = self.forward(x)
        if pred >= 0.5:
            return 1
        else:
            return 0

In [7]:
model = Pima().to(device)
model

Pima(
  (hidden_linear1): Linear(in_features=8, out_features=12, bias=True)
  (activation): ReLU()
  (hidden_linear2): Linear(in_features=12, out_features=8, bias=True)
  (hidden_linear3): Linear(in_features=8, out_features=1, bias=True)
  (output_activation): Sigmoid()
)

In [8]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', ' x '.join(str(x) for x in list(param.size())[::-1]), '=', num_param)
            else:
                print(name, ':', num_param)
                print('-' * 40)
            total_param += num_param
    print('total:', total_param)

In [9]:
count_parameters(model)

hidden_linear1.weight : 8 x 12 = 96
hidden_linear1.bias : 12
----------------------------------------
hidden_linear2.weight : 12 x 8 = 96
hidden_linear2.bias : 8
----------------------------------------
hidden_linear3.weight : 8 x 1 = 8
hidden_linear3.bias : 1
----------------------------------------
total: 221


In [10]:
ds = TensorDataset(X, y)
dataloader = DataLoader(ds, batch_size=5)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.BCELoss()
n_epochs = 100

In [11]:
for epoch in range(n_epochs):
    for data, label in dataloader:
        data = data.type(torch.FloatTensor)
        out = model(data.to(device))
        loss = loss_fn(out, label.type(torch.FloatTensor).unsqueeze(1)).to(device)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.611996
Epoch: 1, Loss: 0.544751
Epoch: 2, Loss: 0.516771
Epoch: 3, Loss: 0.526458
Epoch: 4, Loss: 0.539419
Epoch: 5, Loss: 0.535317
Epoch: 6, Loss: 0.530821
Epoch: 7, Loss: 0.538468
Epoch: 8, Loss: 0.533838
Epoch: 9, Loss: 0.539254
Epoch: 10, Loss: 0.531772
Epoch: 11, Loss: 0.527312
Epoch: 12, Loss: 0.521992
Epoch: 13, Loss: 0.517721
Epoch: 14, Loss: 0.531183
Epoch: 15, Loss: 0.515029
Epoch: 16, Loss: 0.537564
Epoch: 17, Loss: 0.523568
Epoch: 18, Loss: 0.520231
Epoch: 19, Loss: 0.527797
Epoch: 20, Loss: 0.527874
Epoch: 21, Loss: 0.521578
Epoch: 22, Loss: 0.523485
Epoch: 23, Loss: 0.502561
Epoch: 24, Loss: 0.512940
Epoch: 25, Loss: 0.517082
Epoch: 26, Loss: 0.527064
Epoch: 27, Loss: 0.533069
Epoch: 28, Loss: 0.531134
Epoch: 29, Loss: 0.541891
Epoch: 30, Loss: 0.506780
Epoch: 31, Loss: 0.484450
Epoch: 32, Loss: 0.485595
Epoch: 33, Loss: 0.458797
Epoch: 34, Loss: 0.452836
Epoch: 35, Loss: 0.454356
Epoch: 36, Loss: 0.453586
Epoch: 37, Loss: 0.442747
Epoch: 38, Loss: 0.435

In [12]:
train_loader = DataLoader(ds)
correct = 0

with torch.no_grad():
    for data, label in train_loader:
        predicted = model.predict(data.type(torch.FloatTensor).to(device))
        target = int(label[0])
        correct += 1 if predicted == target else 0
    
print("Accuracy: %f" % (correct / len(train_loader.dataset)))

Accuracy: 0.785156
